In [ ]:
import json
import tensorflow as tf

In [ ]:
x = tf.Variable([1, 2], dtype=tf.float32)

In [ ]:
@tf.function
def f(x):
    return tf.square(x)

In [ ]:
with tf.GradientTape() as tape:
    y = f(x)
tape.gradient(y, x)

In [ ]:
x = tf.constant([5, 5], dtype=tf.float16, name='fixed array')

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(x)
    y = f(x)
tape.gradient(y, x)

In [ ]:
x = tf.constant([[1, 1], [5, 5]], dtype=tf.float16, name='fixed array')

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(x)
    y = f(x)
tape.gradient(y, x)

In [ ]:
tf.sign([-5,5])

In [ ]:
@tf.function
def preprocess_input(input_image):
    image = tf.cast(input_image, tf.float32)
    resized_image = tf.image.resize(image, (224, 224))
    return tf.keras.applications.mobilenet_v2.preprocess_input(resized_image)

In [ ]:
def get_model():
    input_layer = tf.keras.layers.Input([None, None, 3], dtype = tf.uint8)
    processed_images = tf.keras.layers.Lambda(preprocess_input)(input_layer)
    output = tf.keras.applications.MobileNetV2(include_top=True, weights='imagenet')(processed_images)
    return tf.keras.Model(inputs=[input_layer], outputs=[output])

In [ ]:
model = get_model()

In [ ]:
image_path = tf.keras.utils.get_file(
    'YellowLabradorLooking_new.jpg', 
    'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'
)
image_raw = tf.io.read_file(image_path)
images = tf.expand_dims(tf.image.decode_image(image_raw), axis=0)

In [ ]:
images.shape, images.dtype

In [ ]:
tf.keras.applications.mobilenet_v2.decode_predictions(
    model.predict(images)
)

In [ ]:
tf.saved_model.save(
    model,
    export_dir='temp',
    signatures=None
)

In [ ]:
def create_adversarial_pattern(input_image, input_label, model):
    loss_function = tf.keras.losses.CategoricalCrossentropy()
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = loss_function(input_label, prediction)

    # Get the gradients of the loss w.r.t the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

In [ ]:
# Get the input label of the image.
labrador_retriever_index = 208
label = tf.expand_dims(
    tf.one_hot(labrador_retriever_index, 1000),
    axis=0
)

In [ ]:
# in order to compute gradients properly, we should preprocess the image before
signed_gradients = create_adversarial_pattern(
    preprocess_input(images),
    label,
    tf.keras.applications.MobileNetV2(include_top=True, weights='imagenet')
)

In [ ]:
tf.keras.applications.mobilenet_v2.decode_predictions(
    tf.keras.applications.MobileNetV2(include_top=True, weights='imagenet').predict(
        preprocess_input(images)
    )
)

In [ ]:
tf.keras.applications.mobilenet_v2.decode_predictions(
    tf.keras.applications.MobileNetV2(include_top=True, weights='imagenet').predict(
        preprocess_input(images) + 1/255*signed_gradients
    )
)